In [1]:
import numpy as np
import pandas as pd
import sklearn.feature_selection as fs
import torch
import clip
from helpers import *
import debias_clip as dclip

def calc_feature_MI(features, labels, n_neighbors = 10, rs=1):
    return fs.mutual_info_classif(features, labels, discrete_features=False, copy=True, n_neighbors=n_neighbors, random_state=rs)

def return_feature_MI_order(features, data, sensitive_attributes, n_neighbors = 10, rs=1):
    labels = data[sensitive_attributes].apply(lambda x: ' '.join(x), axis=1) 
    print(labels)
    feature_MI = calc_feature_MI(features, labels, n_neighbors, rs)
    print(feature_MI)
    feature_order = np.argsort(feature_MI)[::-1]
    print(feature_MI[feature_order])
    return feature_order

In [2]:
data = pd.read_csv('datasets/celeba/list_attr_celeba.csv')
data = data.astype('string')
print(data.head())


MIclip_training = data.iloc[0:20000]
MIclip_training.to_csv('datasets/celeba/MIclip_training.csv')
img_testing = data.iloc[20000:35000]
img_testing.to_csv('datasets/celeba/alg_testing.csv')


     image_id 5_o_Clock_Shadow Arched_Eyebrows Attractive Bags_Under_Eyes  \
0  000001.jpg               -1               1          1              -1   
1  000002.jpg               -1              -1         -1               1   
2  000003.jpg               -1              -1         -1              -1   
3  000004.jpg               -1              -1          1              -1   
4  000005.jpg               -1               1          1              -1   

  Bald Bangs Big_Lips Big_Nose Black_Hair  ... Sideburns Smiling  \
0   -1    -1       -1       -1         -1  ...        -1       1   
1   -1    -1       -1        1         -1  ...        -1       1   
2   -1    -1        1       -1         -1  ...        -1      -1   
3   -1    -1       -1       -1         -1  ...        -1      -1   
4   -1    -1        1       -1         -1  ...        -1      -1   

  Straight_Hair Wavy_Hair Wearing_Earrings Wearing_Hat Wearing_Lipstick  \
0             1        -1                1          -

In [4]:
prefix = 'datasets/celeba/img_align_celeba/'
data['relative_path'] = prefix + data['image_id']    

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device)

batch_size = 200

if True:
    features = process_images(model, preprocess, data['relative_path'], batch_size=batch_size)

np.save('datasets/celeba/features.npy', features)


100%|██████████| 1013/1013 [09:50<00:00,  1.72it/s]


In [4]:
prefix = 'datasets/celeba/img_align_celeba/'
data['relative_path'] = prefix + data['image_id']    

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device)

device_d = "cuda" if torch.cuda.is_available() else "cpu"
model_debias, preprocess_debias = dclip.load("ViT-B/16-gender", device_d)

batch_size = 200

features_debias = process_images(model_debias, preprocess_debias, data['relative_path'], batch_size=batch_size, device=device_d)

device_d = 'cpu'
model_debias, preprocess_debias = dclip.load("ViT-B/16-gender", device_d)

np.save('datasets/celeba/features_debias.npy', features_debias)

Installing pretrained embedings
 best_ndkl_oai-clip-vit-b-16_neptune_run_OXVLB-317_model_e4_step_5334_embeddings.pt...


100%|█████████████████████████████████████| 4.73k/4.73k [00:00<00:00, 4.86MiB/s]
100%|██████████| 1013/1013 [43:38<00:00,  2.58s/it]


Installing pretrained embedings
 best_ndkl_oai-clip-vit-b-16_neptune_run_OXVLB-317_model_e4_step_5334_embeddings.pt...


100%|█████████████████████████████████████| 4.73k/4.73k [00:00<00:00, 4.52MiB/s]


In [6]:
MIfeatures = np.load('datasets/celeba/features.npy')[0:20000]
gender_MI_order = return_feature_MI_order(MIfeatures, MIclip_training, ['Male'])
skintone_MI_order = return_feature_MI_order(MIfeatures, MIclip_training, ['Pale_Skin'])
age_MI_order = return_feature_MI_order(MIfeatures, MIclip_training, ['Young'])
gender_skintone_intersectional_MI_order = return_feature_MI_order(MIfeatures, MIclip_training, ['Male', 'Pale_Skin'])
intersectional_MI_order = return_feature_MI_order(MIfeatures, MIclip_training, ['Pale_Skin', 'Male', 'Young'])
additional_concepts_MI_order = return_feature_MI_order(MIfeatures, MIclip_training, ['Pale_Skin', 'Male', 'Young', 'Attractive'])
                                                       
np.save('datasets/MI_orders/gender.npy', gender_MI_order)
np.save('datasets/MI_orders/skintone.npy', skintone_MI_order)
np.save('datasets/MI_orders/age.npy', age_MI_order)
np.save('datasets/MI_orders/gender_skintone.npy', gender_skintone_intersectional_MI_order)
np.save('datasets/MI_orders/intersectional.npy', intersectional_MI_order)
np.save('datasets/MI_orders/additional_concepts.npy', additional_concepts_MI_order)


0        -1
1        -1
2         1
3        -1
4        -1
         ..
19995    -1
19996    -1
19997    -1
19998    -1
19999     1
Length: 20000, dtype: object


c:\Anaconda\lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


[7.62402647e-03 5.59623347e-03 1.06083289e-02 1.03092231e-02
 9.92813860e-03 1.34443044e-02 6.57873756e-02 1.64915238e-02
 4.17969718e-03 4.72176126e-02 5.29114359e-03 1.90777520e-04
 3.27032282e-02 8.01255590e-03 6.71361250e-02 2.74916736e-03
 1.38839068e-01 6.09242420e-03 2.29844174e-02 4.53173112e-03
 9.44625245e-04 5.00935903e-02 4.95438146e-02 3.18572405e-03
 0.00000000e+00 2.37387020e-02 5.50194568e-03 0.00000000e+00
 9.62180828e-03 7.37917877e-03 2.63016160e-02 1.63941545e-02
 4.55989720e-03 1.73960001e-02 2.28954437e-03 1.64511689e-04
 1.19765449e-03 7.74281141e-02 2.46525749e-03 5.58781630e-02
 7.93609114e-03 3.54887108e-03 5.78152864e-03 4.45097086e-03
 1.24672972e-02 3.33291535e-02 1.05368394e-02 0.00000000e+00
 1.93337929e-02 7.03785400e-03 3.21953096e-03 4.67519082e-02
 0.00000000e+00 3.24903401e-03 1.90179536e-03 3.82695828e-02
 2.12713878e-02 1.90555886e-02 2.75955075e-02 9.21330266e-03
 6.41311385e-02 6.34329958e-03 1.33583740e-02 2.02092027e-02
 8.56734715e-03 1.625245

c:\Anaconda\lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


[0.00000000e+00 0.00000000e+00 3.78017548e-04 5.74759080e-04
 3.30401654e-03 0.00000000e+00 4.49232266e-03 1.90976439e-03
 2.16217163e-03 0.00000000e+00 0.00000000e+00 0.00000000e+00
 2.72821356e-03 0.00000000e+00 1.91612691e-03 9.25818358e-04
 4.08095646e-03 3.03313959e-04 4.19937919e-04 2.03696854e-03
 1.23170563e-03 5.84049112e-04 5.25071635e-05 0.00000000e+00
 9.15083526e-04 9.33691562e-04 1.61543877e-03 0.00000000e+00
 0.00000000e+00 0.00000000e+00 5.71290593e-04 0.00000000e+00
 5.03193053e-05 0.00000000e+00 0.00000000e+00 1.83265778e-04
 0.00000000e+00 4.42359170e-03 0.00000000e+00 1.77221193e-03
 0.00000000e+00 1.04740355e-03 2.57428642e-03 9.61422927e-04
 3.14373405e-03 1.97697956e-03 1.89710655e-03 1.17721701e-03
 0.00000000e+00 2.26417137e-03 4.48179669e-05 3.73960717e-03
 9.46251286e-04 9.16944736e-04 1.20470212e-03 7.82532245e-04
 2.67903099e-03 1.30730630e-03 5.68593718e-04 1.69069833e-03
 1.88257672e-03 8.35564630e-04 9.41610731e-04 5.78611586e-03
 3.65236583e-04 7.322011

...

c:\Anaconda\lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


[4.72207887e-02 1.60048679e-02 1.24777740e-02 3.35192478e-02
 3.60202276e-02 2.86765955e-02 8.22692175e-02 2.27107492e-02
 1.01528408e-02 6.01356583e-02 1.60922395e-02 1.48340029e-02
 4.27000094e-02 1.55478647e-02 9.24434583e-02 6.04814131e-03
 1.48404363e-01 9.86442569e-03 7.84786756e-02 4.43422186e-02
 1.46010230e-02 5.22948429e-02 5.87580301e-02 9.03325078e-03
 4.67251528e-03 3.12406077e-02 3.54985096e-02 1.08741985e-02
 8.35122863e-03 1.35241203e-02 3.12542431e-02 3.35283216e-02
 8.97719662e-03 2.33216634e-02 2.39635496e-02 3.96653097e-04
 0.00000000e+00 8.10107710e-02 6.01487521e-03 6.46887427e-02
 1.55932417e-02 2.38174075e-02 1.98271746e-02 1.55210444e-02
 2.01476364e-02 6.84710784e-02 1.93002041e-02 9.18399213e-03
 3.25894969e-02 1.59802677e-02 9.64220429e-03 6.28985449e-02
 0.00000000e+00 3.30699817e-02 2.04324359e-02 6.76878966e-02
 4.76148683e-02 2.11401162e-02 3.58522215e-02 1.01391470e-02
 8.83343790e-02 1.69243288e-02 7.09515608e-02 3.95268409e-02
 1.23130660e-02 2.854869

c:\Anaconda\lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


[0.05257782 0.02517811 0.02363589 0.0384449  0.06096189 0.04515066
 0.08613857 0.02556748 0.02412607 0.07031388 0.02732449 0.02596225
 0.0531932  0.01885934 0.10731801 0.00790014 0.16482383 0.01203806
 0.09495235 0.04266093 0.01631953 0.05632779 0.06055015 0.02654561
 0.00479313 0.03308327 0.03697528 0.02113168 0.01047241 0.01730798
 0.0326734  0.03944067 0.013846   0.02781421 0.02351717 0.00178294
 0.00490385 0.08432714 0.01075125 0.07465247 0.01484944 0.03218335
 0.01869914 0.01673715 0.02443534 0.07583924 0.02164136 0.00995415
 0.03198889 0.02112982 0.00925805 0.07002435 0.00114772 0.04007052
 0.02375893 0.06629534 0.05065648 0.02260729 0.04107317 0.01212375
 0.0959688  0.01928688 0.07779841 0.04448165 0.02293274 0.02958208
 0.0257808  0.50799526 0.01395946 0.02530341 0.0559288  0.02520327
 0.01248186 0.03723681 0.02634201 0.02163642 0.03692266 0.02474712
 0.0430392  0.03135369 0.01742078 0.06099091 0.04713771 0.09704178
 0.01714108 0.00630017 0.06455899 0.0209269  0.0892982  0.0201

AttributeError: 'numpy.ndarray' object has no attribute 'npy'

In [9]:
print(age_MI_order)

[357  93 106 198 157 401 210 367 404 311 117 376 218 321 247 503 282  18
 493 177 152 298  67 286 410  83 277  55  14 419 356 296 355 190 415  62
   6 428  19 183 181 389  96  26 446 184   4 327 272 491 284 245  16  76
 147  91 290 408 122 334 100  98   9 310  53 295  73 352 375 470 465 431
 480   5 285 385 451 116 113 391  86 324  79 226 217 303  22 115 266 315
 158 163 105 227  56 442 164   3 453 383 366 377 390 432 379 434 462 235
 279 292  71 241 267 148 283 392 170  37  54 212 308 172 203 137 423  41
  27 381 305 328 109 123 250 121 338 501 107 110 450 209   1  21  34  45
 329 489 425 464 457  10 242 511 384   0 168 497 127 506 300 448 374 301
  94 254  30 347 359 222 500 323  20 194 229 258 103  12  31 490 368 182
  75  13 175 230 270  61 120 444 402 488 350 508 443 149  66 278 510 143
 102 331 221 471  77 405 496 387 380 335 129 378  29 441 104 134 483   2
 244 195 240  60  65 274 233 119 179 364  23 474 468 306 200   7  89 461
 202 187 201 252 293 231 504 228 193 260 206 317 29